# Import module

In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
import sys

from importlib import reload


# sys.path.append("D:/DST/")
import src.cathay_db as db
import src.utils as ut
import src.financial_statement as fs

reload(ut)
reload(fs)
import matplotlib.pyplot as plt
import datetime
import os

# set max display rows
pd.set_option("display.max_rows", 200)

# set max display columns
pd.set_option("display.max_columns", 100)

# Set the float format to display without scientific notation
pd.options.display.float_format = '{:.4f}'.format

# 1. Read data from feather file

In [2]:
# # following is data from tej
# df_adjusted_price = pd.read_feather('data/tej_adjusted_price.feather')
# df_cash_dividends = pd.read_feather('data/tej_cash_dividends.feather')
# df_shareholder_meeting = pd.read_feather('data/tej_shareholder_meeting.feather')
# df_capital_formation = pd.read_feather('data/tej_capital_formation.feather')

# # following is data from cmoney
# df_price = pd.read_feather('data/cmoney_price.feather')
# df_company_info = pd.read_feather('data/cmoney_company_info.feather')
# df_financial_statement = pd.read_feather('data/cmoney_financial_statement.feather')

# ticker_list = np.sort(pd.unique(df_company_info['股票代號']))

In [3]:
# df_price = ut.cmoney_data_clean_up(df_price)
# df_financial_statement = ut.cmoney_data_clean_up(df_financial_statement)

# df_price = deepcopy(df_price[df_price['股票代號'].isin(ticker_list)])
# df_financial_statement = deepcopy(df_financial_statement[df_financial_statement['股票代號'].isin(ticker_list)])

# df_adjusted_price = ut.tej_data_clean_up(df_adjusted_price)
# df_cash_dividends = ut.tej_data_clean_up(df_cash_dividends)
# df_shareholder_meeting = ut.tej_data_clean_up(df_shareholder_meeting)
# df_capital_formation = ut.tej_data_clean_up(df_capital_formation)

# df_adjusted_price = deepcopy(df_adjusted_price[df_adjusted_price['股票代號'].isin(ticker_list)])
# df_cash_dividends = deepcopy(df_cash_dividends[df_cash_dividends['股票代號'].isin(ticker_list)])
# df_shareholder_meeting = deepcopy(df_shareholder_meeting[df_shareholder_meeting['股票代號'].isin(ticker_list)])
# df_capital_formation = deepcopy(df_capital_formation[df_capital_formation['股票代號'].isin(ticker_list)])

In [4]:
# # save upper df to feather
# df_price = df_price.reset_index(drop=True)
# df_price.to_feather('data/df_price.feather')
# df_company_info.to_feather('data/df_company_info.feather')

# duplicates = df_financial_statement.columns[df_financial_statement.columns.duplicated()]
# new_columns = [f"{col}_{i}" if col in duplicates else col for i, col in enumerate(df_financial_statement.columns)]
# df_financial_statement.columns = new_columns

# df_financial_statement = df_financial_statement.reset_index(drop=True)
# df_financial_statement.to_feather('data/df_financial_statement.feather')

# df_adjusted_price = df_adjusted_price.reset_index(drop=True)
# df_adjusted_price.to_feather('data/df_adjusted_price.feather')

# df_cash_dividends = df_cash_dividends.reset_index(drop=True)
# df_cash_dividends.to_feather('data/df_cash_dividends.feather')

# df_shareholder_meeting = df_shareholder_meeting.reset_index(drop=True)
# df_shareholder_meeting.to_feather('data/df_shareholder_meeting.feather')

# df_capital_formation = df_capital_formation.reset_index(drop=True)
# df_capital_formation.to_feather('data/df_capital_formation.feather')

In [5]:
# read from feather
df_price = pd.read_feather('data/df_price.feather')
df_company_info = pd.read_feather('data/df_company_info.feather')
# df_financial_statement = pd.read_feather('data/df_tej_fs_slice_cum.feather')
df_adjusted_price = pd.read_feather('data/df_adjusted_price.feather')
df_cash_dividends = pd.read_feather('data/df_cash_dividends.feather')
df_shareholder_meeting = pd.read_feather('data/df_shareholder_meeting.feather')
df_capital_formation = pd.read_feather('data/df_capital_formation.feather')
ticker_list = np.sort(pd.unique(df_company_info['股票代號']))

In [6]:
df_financial_statement = pd.read_feather('data/df_tej_fs_20230712.feather')
df_financial_statement = ut.tej_data_clean_up(df_financial_statement)
df_financial_statement.columns = [x.strip() for x in df_financial_statement.columns]
df_financial_statement['年季'] = df_financial_statement['年月'].apply(lambda x: x.strftime('%Y') + str(int(x.strftime('%m'))//3).zfill(2))
df_financial_statement.sort_values(['股票代號', '年月'], inplace=True)
# df_financial_statement.columns = ['股票名稱', '股票代號', '公司', '年月', '季別', '普通股股本', '特別股股本', '不動產廠房及設備',
#        '財報年月迄日', '財報年月起日', '財報類別（1個別2個體3合併）', '建立日期', '長短期借款', '資產總額',
#        '股東權益總額', '來自營運之現金流量', 'ni', '年季']

df_financial_statement['建立日期'] = pd.to_datetime(df_financial_statement['財報發布日'])
df_financial_statement = df_financial_statement[df_financial_statement['股票代號'].isin(ticker_list)]

column is already exist


In [7]:
# read data from feather which is from main_01_data_cleaning.ipynb
df_price_liquidity_flag = pd.read_feather('./data/df_price_liquidity_flag.feather')
df_book_capital_melt = pd.read_feather('./data/df_book_capital_melt.feather')
df_price_dividend_slice = pd.read_feather('./data/df_price_dividend_slice.feather')
df_price_dividend = pd.read_feather('./data/df_price_dividend.feather')
df_share_outstanding = pd.read_feather('./data/df_share_outstanding.feather')

In [8]:
ticker_list = np.sort(pd.unique(df_company_info['股票代號']))

# Q5 Factors: 1. Market Factor
## 1.1 Market Factor: Stock Excess Return

In [9]:
df_risk_free_rate = pd.read_excel('./data/risk_free_rate.xlsx')

# change the column name to english
df_risk_free_rate.columns = ['date', 'risk_free_rate']

# change the date format, from 2023M05 to datetime format YYYY-MM
df_risk_free_rate['date'] = df_risk_free_rate['date'].apply(lambda x: datetime.datetime.strptime(x, '%YM%m').strftime('%Y-%m'))

# shift the date to next month and create a data valid date for it
df_risk_free_rate['data_valid_date'] = df_risk_free_rate['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m') + datetime.timedelta(days=31))

# sort the data_valid_date for ffill
df_risk_free_rate = df_risk_free_rate.sort_values(by=['data_valid_date']).reset_index(drop=True)

# divide the risk_free_rate by 100
df_risk_free_rate['risk_free_rate'] = df_risk_free_rate['risk_free_rate'] / 100

# select the data_valid_date and risk_free_rate
df_risk_free_rate = df_risk_free_rate[['data_valid_date', 'risk_free_rate']]

# change the data to daily data
df_risk_free_rate = df_risk_free_rate.set_index('data_valid_date').resample('D').ffill()

# change the annualized risk_free_rate to monthly risk_free_rate
df_risk_free_rate['risk_free_rate'] = (1 + df_risk_free_rate['risk_free_rate']) ** (1/12) - 1

In [10]:
df_excess_return = df_adjusted_price[['年月日', '股票代號', '股票名稱', '收盤價(元)']].reset_index(drop=True).copy()
# change the column name
df_excess_return.columns = ['date', 'ticker', 'name', 'price']

# change the price to float
df_excess_return['price'] = df_excess_return['price'].astype(float)

# groupby the ticker and calculate 20 days return by pct_change
df_excess_return['20_d_return'] = df_excess_return.groupby('ticker')['price'].pct_change(20)

# merge the risk_free_rate to df_excess_return
df_excess_return = df_excess_return.merge(df_risk_free_rate, how='left', left_on='date', right_on='data_valid_date')

# calculate the excess return by 20 days return - risk_free_rate
df_excess_return['excess_return'] = df_excess_return['20_d_return'] - df_excess_return['risk_free_rate']

df_excess_return.dropna(inplace=True)

df_excess_return = df_excess_return.reset_index(drop=True)

# fs.CDF(df_excess_return).find_ticker('2330').tail(30)

## 1.2 Investment Factor:
### 1.2.1 Investment Factor: Asset Growth

In [11]:
df_asset_melt = fs.quarterly_to_daily(df_financial_statement[df_financial_statement['股票代號'].isin(ticker_list)], '資產總額')
df_asset_melt.columns = ['date', 'ticker', 'asset']
# # calculate the yoy and qoq
# df_asset_melt['asset_yoy'] = df_asset_melt.groupby('ticker', group_keys=False, as_index=False)['asset'].pct_change(252)
# df_asset_melt['asset_qoq'] = df_asset_melt.groupby('ticker', group_keys=False, as_index=False)['asset'].pct_change(63)

df_asset_qoq = fs.calc_periodic(df_financial_statement, '資產總額','qoq')
df_asset_yoy = fs.calc_periodic(df_financial_statement, '資產總額','yoy')

df_asset_qoq_melt = fs.quarterly_to_daily(df_asset_qoq, '資產總額_qoq')
df_asset_yoy_melt = fs.quarterly_to_daily(df_asset_yoy, '資產總額_yoy')

### 1.2.2 Investment Factor: PPE Growth

In [12]:
df_financial_statement.columns

Index(['股票名稱', '股票代號', '公司', '年月', '季別', '普通股股本', '特別股股本', '不動產廠房及設備',
       '財報年月迄日', '財報年月起日', '財報類別（1個別2個體3合併）', '財報發布日', '長短期借款', '資產總額',
       '股東權益總額', '來自營運之現金流量', 'ni', '營業收入淨額', '營業利益', '年季', '建立日期'],
      dtype='object')

In [13]:
# df_ppe_melt = fs.quarterly_to_daily(df_financial_statement[df_financial_statement['股票代號'].isin(ticker_list)], '不動產廠房及設備')
# df_ppe_melt.columns = ['date', 'ticker', 'ppe']
# # calculate the yoy and qoq
# df_ppe_melt['ppe_yoy'] = df_ppe_melt.groupby('ticker', group_keys=False, as_index=False)['ppe'].pct_change(252)
# df_ppe_melt['ppe_qoq'] = df_ppe_melt.groupby('ticker', group_keys=False, as_index=False)['ppe'].pct_change(63)

df_ppe_qoq = fs.calc_periodic(df_financial_statement, '不動產廠房及設備','qoq')
df_ppe_qoq_melt = fs.quarterly_to_daily(df_ppe_qoq, '不動產廠房及設備_qoq')
df_ppe_yoy = fs.calc_periodic(df_financial_statement, '不動產廠房及設備','yoy')
df_ppe_yoy_melt = fs.quarterly_to_daily(df_ppe_yoy, '不動產廠房及設備_yoy')

## 1.3 Profitability Factor: ROE

In [14]:
df_ni_qoq = fs.calc_periodic(df_financial_statement, 'ni','qoq')
df_ni_qoq_melt = fs.quarterly_to_daily(df_ni_qoq, 'ni_qoq')
df_ni_yoy = fs.calc_periodic(df_financial_statement, 'ni','yoy')
df_ni_yoy_melt = fs.quarterly_to_daily(df_ni_yoy, 'ni_yoy')

In [ ]:
df_ni_melt = fs.quarterly_to_daily(df_financial_statement, 'ni')
df_ni_melt.columns = ['date', 'ticker', 'ni']
# calculate the yoy and qoq
# df_ni_melt['ni_yoy'] = df_ni_melt.groupby('ticker', group_keys=False, as_index=False)['ni'].pct_change(252)
# df_ni_melt['ni_qoq'] = df_ni_melt.groupby('ticker', group_keys=False, as_index=False)['ni'].pct_change(63)

df_preferred_share_melt = fs.quarterly_to_daily(df_financial_statement, '特別股股本')
df_preferred_share_melt.columns = ['date', 'ticker', 'preferred_share']

df_equity_melt = fs.quarterly_to_daily(df_financial_statement, '股東權益總額')
df_equity_melt.columns = ['date', 'ticker', 'equity']

df_equity_melt = df_equity_melt.merge(df_preferred_share_melt, how='left', on=['date', 'ticker'])
df_equity_melt['preferred_share'] = df_equity_melt['preferred_share'].fillna(0)
df_equity_melt['equity - preferred_share'] = df_equity_melt['equity'] - df_equity_melt['preferred_share']
df_roe_melt = df_ni_melt.merge(df_equity_melt[['date', 'ticker', 'equity - preferred_share']], how='left', on=['date', 'ticker']).copy()
df_roe_melt['roe'] = df_roe_melt['ni'] / (df_roe_melt['equity - preferred_share'] / 4)

In [ ]:
df_ni_4q_sum = fs.flow_item_to_quarter_data(df_financial_statement, 'ni')
df_ni_4q_sum_daily = fs.quarterly_to_daily(df_ni_4q_sum,'ni_4q_sum')
# df_opm = df_operating_profit.merge(df_net_sales, how='left', on=['年季', '股票代號', '建立日期'])
# df_opm['opm'] = df_opm['營業利益_4q_sum'] / df_opm['營業收入淨額_4q_sum']
# df_opm_daily = fs.quarterly_to_daily(df_opm,'opm')

In [ ]:
accounting_item = 'ni'

df = df_financial_statement[['年季','股票代號','建立日期', accounting_item]].copy()

# drop na
df.dropna(inplace=True)

# drop the duplicate
df.drop_duplicates(inplace=True)

df = df.sort_values(by=["股票代號", "年季"]).reset_index(drop=True)

df[f"{accounting_item}"] = df[f"{accounting_item}"].astype(float)

df["季底日期"] = df["年季"].apply(lambda x: fs.quarter_end_date(x))

# Set '建立日期' as the index of the dataframe

# count the number of quarter between '財報年月起日' and '財報年月迄日' by minus the end quarter and start quarter
# # turn 財報年月起日 to yyyy/mm and 財報年月迄日 to yyyy/mm
# df['年季_dt'] = df['年季'].apply(lambda x: datetime.datetime.strptime(x, '%Y%m'))
df['季底日期_last']= df.groupby('股票代號', as_index=False, group_keys=False)['季底日期'].shift(1)
df['季底日期_next']= df.groupby('股票代號', as_index=False, group_keys=False)['季底日期'].shift(-1)
# calculate the date difference between '年季_last' and '年季_next' with '年季_dt'
df['季底日期_next_dt'] = df['季底日期_next'] - df['季底日期']
df['季底日期_dt_last'] = df['季底日期'] - df['季底日期_last']
df['季底日期_max'] = df[['季底日期_dt_last', '季底日期_next_dt']].max(axis=1)
df['季底日期_max'] = df['季底日期_max'].apply(lambda x: x.days)

# if 季底日期_max is smaller than 100, then it is a quarterly data, if greater than 200, then it is a yearly data, if between 100 and 200, then it is a half yearly data
df['財報類別'] = df['季底日期_max'].apply(lambda x: 1 if x < 100 else 4 if x > 200 else 2)
df.set_index("季底日期", inplace=True)

df_grouped = df.groupby("股票代號")[accounting_item].resample("Q").mean().reset_index()
df.reset_index(inplace=True)
df = df_grouped.merge(df[['股票代號', '季底日期', '財報類別','建立日期']], how='left', on=['股票代號', '季底日期']).copy()

# turn the '季底日期' to '年季'
df["年季"] = df["季底日期"].apply(
    lambda x: x.strftime("%Y") + str(int(x.strftime("%m")) // 3).zfill(2)
)

df["建立日期"] = df.groupby("股票代號")["建立日期"].bfill()
df.fillna(0, inplace=True)

df[f'{accounting_item}_4_row_sum'] = df.groupby('股票代號', as_index=False, group_keys=False).apply(lambda x: x[accounting_item].rolling(4).sum())

df[f'number_of_quarter_sum'] = df.groupby('股票代號', as_index=False, group_keys=False).apply(lambda x: x['財報類別'].rolling(4).sum())

df[f'{accounting_item}_4q_sum'] = df[f'{accounting_item}_4_row_sum'] / df[f'number_of_quarter_sum'] * 4


df = ut.tej_data_clean_up(df)
df_ni_4q_sum_melt = fs.quarterly_to_daily(df, 'ni_4q_sum')
df_ni_4q_sum_melt.columns = ['date', 'ticker', 'ni_4q_sum']

df_roe_4q_sum_melt = df_ni_4q_sum_melt.merge(df_equity_melt[['date', 'ticker', 'equity - preferred_share']], how='left', on=['date', 'ticker']).copy()
df_roe_4q_sum_melt['roe_4q_sum'] = df_roe_4q_sum_melt['ni_4q_sum'] / df_roe_4q_sum_melt['equity - preferred_share'] 

In [ ]:
df_ni_4q_sum_qoq = fs.calc_periodic(df, 'ni_4q_sum','qoq')
df_ni_4q_sum_qoq_melt = fs.quarterly_to_daily(df_ni_4q_sum_qoq, 'ni_4q_sum_qoq')


df_ni_4q_sum_yoy = fs.calc_periodic(df, 'ni_4q_sum','yoy')
df_ni_4q_sum_yoy_melt = fs.quarterly_to_daily(df_ni_4q_sum_yoy, 'ni_4q_sum_yoy')

## 1.4 Tobin's Q

In [ ]:
df_market_cap_melt = pd.merge(df_price[['日期','股票代號','收盤價']], df_share_outstanding, how='left', left_on=['日期','股票代號'], right_on=['日期','股票代號'])
df_market_cap_melt['收盤價'] = df_market_cap_melt['收盤價'].astype(float)
df_market_cap_melt['流通在外股數'] = df_market_cap_melt['流通在外股數'].astype(float)
df_market_cap_melt.columns = ['date', 'ticker', 'price', 'share_outstanding']

# the market cap is the product of price and share outstanding * 1000(share) / 1000(unit)
df_market_cap_melt['market_cap'] = df_market_cap_melt['price'] * df_market_cap_melt['share_outstanding']
df_market_cap_melt.sort_values(by=['ticker', 'date'], inplace=True)
df_market_cap_melt = df_market_cap_melt.reset_index(drop=True)

df_long_short_term_liability_melt = fs.quarterly_to_daily(df_financial_statement[df_financial_statement['股票代號'].isin(ticker_list)], '長短期借款')
df_long_short_term_liability_melt.columns = ['date', 'ticker', 'long_short_term_liability']


df_tobins_q_melt = df_market_cap_melt.merge(df_asset_melt, how='left', on=['date', 'ticker']).merge(df_long_short_term_liability_melt, how='left', on=['date', 'ticker'])
df_tobins_q_melt['long_short_term_liability'] = df_tobins_q_melt['long_short_term_liability'].fillna(0)


df_tobins_q_melt['tobins_q'] = (df_tobins_q_melt['market_cap'] + df_tobins_q_melt['long_short_term_liability'] ) / df_tobins_q_melt['asset']

## 1.5 OCF

In [ ]:
df_ocf_melt = fs.quarterly_to_daily(df_financial_statement, '來自營運之現金流量')
df_ocf_melt.columns = ['date', 'ticker', 'ocf']

# merge asset to df_ocf_melt
df_ocf_melt = df_ocf_melt.merge(df_asset_melt, how='left', on=['date', 'ticker'])
df_ocf_melt['ocf / asset'] = df_ocf_melt['ocf'] / df_ocf_melt['asset']

## 1.6 dROE

In [ ]:
df_roe_melt['roe_yoy'] = df_roe_melt.groupby('ticker', group_keys=False, as_index=False)['roe'].pct_change(252)
 
df_roe_4q_sum_melt['roe_4q_sum_yoy'] = df_roe_4q_sum_melt.groupby('ticker', group_keys=False, as_index=False)['roe_4q_sum'].pct_change(252)

## 1.7 OPM

In [ ]:
df_net_sales = fs.flow_item_to_quarter_data(df_financial_statement, '營業收入淨額')
df_operating_profit = fs.flow_item_to_quarter_data(df_financial_statement, '營業利益')
df_opm = df_operating_profit.merge(df_net_sales, how='left', on=['年季', '股票代號', '建立日期'])
df_opm['opm'] = df_opm['營業利益_4q_sum'] / df_opm['營業收入淨額_4q_sum']
df_opm_daily = fs.quarterly_to_daily(df_opm,'opm')

In [ ]:
df_opm_daily_qoq = df_opm_daily.groupby('股票代號', group_keys=False, as_index=False)['opm'].pct_change(62)
df_opm_daily_yoy = df_opm_daily.groupby('股票代號', group_keys=False, as_index=False)['opm'].pct_change(252)

# Factor df to feather file 

In [ ]:
df_excess_return.to_feather('./data/df_excess_return.feather')
df_asset_qoq_melt.to_feather('./data/df_asset_qoq_melt.feather')
df_asset_yoy_melt.to_feather('./data/df_asset_yoy_melt.feather')
df_ppe_qoq_melt.to_feather('./data/df_ppe_qoq_melt.feather')
df_ppe_yoy_melt.to_feather('./data/df_ppe_yoy_melt.feather')
df_ni_qoq_melt.to_feather('./data/df_ni_qoq_melt.feather')
df_ni_yoy_melt.to_feather('./data/df_ni_yoy_melt.feather')
df_ni_4q_sum_qoq_melt.to_feather('./data/df_ni_4q_sum_qoq_melt.feather')
df_ni_4q_sum_yoy_melt.to_feather('./data/df_ni_4q_sum_yoy_melt.feather')
df_roe_melt.to_feather('./data/df_roe_melt.feather')
df_roe_4q_sum_melt.to_feather('./data/df_roe_4q_sum_melt.feather')
df_tobins_q_melt.to_feather('./data/df_tobins_q_melt.feather')
df_ocf_melt.to_feather('./data/df_ocf_melt.feather')
df_market_cap_melt.to_feather('./data/df_market_cap_melt.feather')

In [ ]:
df_market_cap_melt.to_feather('./data/df_market_cap_melt.feather')

In [ ]:
# read data from feather which is from main_02_factor.ipynb
df_adjusted_price = pd.read_feather('data/df_adjusted_price.feather')
df_excess_return = pd.read_feather('./data/df_excess_return.feather')
df_asset_qoq_melt = pd.read_feather('./data/df_asset_qoq_melt.feather')
df_asset_yoy_melt = pd.read_feather('./data/df_asset_yoy_melt.feather')
df_ppe_qoq_melt = pd.read_feather('./data/df_ppe_qoq_melt.feather')
df_ppe_yoy_melt = pd.read_feather('./data/df_ppe_yoy_melt.feather')
df_ni_qoq_melt = pd.read_feather('./data/df_ni_qoq_melt.feather')
df_ni_yoy_melt = pd.read_feather('./data/df_ni_yoy_melt.feather')
df_ni_4q_sum_qoq_melt = pd.read_feather('./data/df_ni_4q_sum_qoq_melt.feather')
df_ni_4q_sum_yoy_melt = pd.read_feather('./data/df_ni_4q_sum_yoy_melt.feather')
df_roe_melt = pd.read_feather('./data/df_roe_melt.feather')
df_roe_4q_sum_melt = pd.read_feather('./data/df_roe_4q_sum_melt.feather')
df_tobins_q_melt = pd.read_feather('./data/df_tobins_q_melt.feather')
df_ocf_melt = pd.read_feather('./data/df_ocf_melt.feather')
df_market_cap_melt = pd.read_feather('./data/df_market_cap_melt.feather')

# Merge the factors to one df

In [ ]:
df_factor_all = df_adjusted_price[['年月日','股票代號','收盤價(元)']].copy()
df_factor_all.columns = ['date', 'ticker', 'price']
# change the date to datetime format
df_factor_all['date'] = pd.to_datetime(df_factor_all['date'], format='%Y%m%d')
# change the price to float
df_factor_all['price'] = df_factor_all['price'].astype(float)

In [ ]:
# merge the df_excess_return to df_factor_all
df_factor_all = df_factor_all.merge(df_excess_return[['date', 'ticker', 'excess_return']], how='left', on=['date', 'ticker'])

df_asset_qoq_melt.columns = ['date', 'ticker', 'asset_qoq']
# merge the factor df to df_factor_all
df_factor_all = pd.merge(df_factor_all, df_asset_qoq_melt, how='left', on=['date', 'ticker'])

df_asset_yoy_melt.columns = ['date', 'ticker', 'asset_yoy']
# merge the factor df to df_factor_all
df_factor_all = pd.merge(df_factor_all, df_asset_yoy_melt, how='left', on=['date', 'ticker'])

df_ppe_qoq_melt.columns = ['date', 'ticker', 'ppe_qoq']
# merge the factor df to df_factor_all
df_factor_all = pd.merge(df_factor_all, df_ppe_qoq_melt, how='left', on=['date', 'ticker'])

df_ppe_yoy_melt.columns = ['date', 'ticker', 'ppe_yoy']
# merge the factor df to df_factor_all
df_factor_all = pd.merge(df_factor_all, df_ppe_yoy_melt, how='left', on=['date', 'ticker'])

df_ni_qoq_melt.columns = ['date', 'ticker', 'ni_qoq']
# merge the factor df to df_factor_all
df_factor_all = pd.merge(df_factor_all, df_ni_qoq_melt, how='left', on=['date', 'ticker'])

df_ni_yoy_melt.columns = ['date', 'ticker', 'ni_yoy']
# merge the factor df to df_factor_all
df_factor_all = pd.merge(df_factor_all, df_ni_yoy_melt, how='left', on=['date', 'ticker'])

df_roe_melt = df_roe_melt[['date', 'ticker', 'roe', 'roe_yoy']].copy()
# merge the factor df to df_factor_all
df_factor_all = pd.merge(df_factor_all, df_roe_melt, how='left', on=['date', 'ticker'])

df_roe_4q_sum_melt = df_roe_4q_sum_melt[['date', 'ticker', 'roe_4q_sum', 'roe_4q_sum_yoy']].copy()
# merge the factor df to df_factor_all
df_factor_all = pd.merge(df_factor_all, df_roe_4q_sum_melt, how='left', on=['date', 'ticker'])

df_tobins_q_melt = df_tobins_q_melt[['date', 'ticker', 'tobins_q']].copy()
# merge the factor df to df_factor_all
df_factor_all = pd.merge(df_factor_all, df_tobins_q_melt, how='left', on=['date', 'ticker'])

df_ocf_melt = df_ocf_melt[['date', 'ticker', 'ocf / asset']].copy()
# merge the factor df to df_factor_all
df_factor_all = pd.merge(df_factor_all, df_ocf_melt, how='left', on=['date', 'ticker'])

df_market_cap_melt = df_market_cap_melt[['date', 'ticker', 'market_cap']].copy()
# merge the factor df to df_factor_all
df_factor_all = pd.merge(df_factor_all, df_market_cap_melt, how='left', on=['date', 'ticker'])

In [ ]:
df_factor_all['20_d_return'] = df_factor_all.groupby('ticker',as_index=False, group_keys=False)['price'].apply(lambda x: x.pct_change(20))
df_factor_all['40_d_return'] = df_factor_all.groupby('ticker',as_index=False, group_keys=False)['price'].apply(lambda x: x.pct_change(40))
df_factor_all['60_d_return'] = df_factor_all.groupby('ticker',as_index=False, group_keys=False)['price'].apply(lambda x: x.pct_change(60))

In [ ]:
df_price_dividend['dividend_1Y_sum_yield'] = df_price_dividend['DIV_fillna_0_1_Y_sum'] / df_price_dividend['收盤價']
df_price_dividend['dividend_2Y_sum_yield'] = df_price_dividend['DIV_fillna_0_2_Y_sum'] / df_price_dividend['收盤價']
df_price_dividend['dividend_3Y_sum_yield'] = df_price_dividend['DIV_fillna_0_3_Y_sum'] / df_price_dividend['收盤價']

df_price_dividend = df_price_dividend[['日期','股票代號','dividend_1Y_sum_yield','dividend_2Y_sum_yield','dividend_3Y_sum_yield']].copy()

df_price_dividend.columns = ['date', 'ticker', 'dividend_1Y_sum_yield', 'dividend_2Y_sum_yield', 'dividend_3Y_sum_yield']

df_factor_all = pd.merge(df_factor_all, df_price_dividend, how='left', on=['date', 'ticker'])

In [ ]:
df_price_dividend_slice.columns = ['date', 'ticker', 'last_dividend_yield']

df_factor_all = pd.merge(df_factor_all, df_price_dividend_slice, how='left', on=['date', 'ticker'])

In [ ]:
df_factor_all.sort_values(by=['ticker', 'date'], inplace=True)

In [ ]:
df_factor_all = df_factor_all.reset_index(drop=True)
df_factor_all.to_feather('./data/df_factor_all.feather')